In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import networkx as nx
import os
from sklearn.metrics import mutual_info_score
from scipy.stats import zscore
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#import packages
import pandas as pd
import wget
from datetime import date
import gzip
import pandas as pd
from itertools import combinations

In [67]:
drug_response = pd.read_csv("data/drug_response.csv", index_col=0)
drug_response = drug_response[drug_response['TCGA_DESC'] != 'UNCLASSIFIED']
# Count occurrences of each value in 'TCGA_DESC'
value_counts = drug_response['TCGA_DESC'].value_counts()
# Filter to keep only those values that appear more than 10,000 times
values_to_keep = value_counts[value_counts > 10000].index
# Filter the DataFrame to include only rows where 'TCGA_DESC' is in the values_to_keep
drug_response = drug_response[drug_response['TCGA_DESC'].isin(values_to_keep)]
models = drug_response.index.drop_duplicates()
len(models)

634